In [41]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
import time


In [42]:
df = pd.read_csv('../data/clean_data_state_response.csv')

In [43]:
df.head()

,protestnumber,protesterviolence,participants,notes,accomodation,arrests,beatings,crowd dispersal,ignore,killings,...,year_2017,year_2018,year_2019,region_Asia,region_Central America,region_Europe,region_MENA,region_North America,region_Oceania,region_South America
0,1,0.0,3000,Canada s railway passenger system was finally...,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,2,0.0,1000,protestors were only identified as young peop...,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,3,0.0,500,"THE Queen, after calling on Canadians to rema...",0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,4,1.0,300,Canada s federal government has agreed to acq...,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,5,1.0,950,Protests were directed against the state due t...,1,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [44]:
import time
import datetime
def datetime_to_int(dt):
    return int(dt.strftime("%Y%m%d%H%M%S"))

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14474 entries, 0 to 14473
Columns: 222 entries, protestnumber to region_South America
dtypes: float64(1), int64(219), object(2)
memory usage: 24.5+ MB


# Text Cleaning

In [46]:
import nltk
from nltk.stem import WordNetLemmatizer
import string

def tokenize(text):
    text = ''.join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]
                                

In [47]:
tf_vectorizer = TfidfVectorizer(tokenizer=tokenize,
                                stop_words = 'english', 
                                lowercase = True,
                                ngram_range = (1,2),
                                max_df = 0.5, 
                                min_df = 10,
                                max_features = 3000) 



In [48]:
df['notes'] = df['notes'].str.replace(r'(\b\w{1,2}\b)', '') # for words removing 1 or 2 letter words
 


In [49]:
notes=df['notes']


In [50]:
tf_vectorizer.fit(notes)

/Users/adrianchapman/Dev/virtualenvs/py3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
/Users/adrianchapman/Dev/virtualenvs/py3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


TfidfVectorizer(max_df=0.5, max_features=3000, min_df=10, ngram_range=(1, 2),
                stop_words='english',
                tokenizer=<function tokenize at 0x13917d430>)

In [51]:
notes_vc=tf_vectorizer.transform(notes)

In [52]:
tf_vectorizer.get_feature_names()[:200]

['000',
 '000 000',
 '000 demonstrator',
 '000 people',
 '000 police',
 '000 protester',
 '000 student',
 '000 supporter',
 '000 worker',
 '000s',
 '100',
 '100 people',
 '1000',
 '100000',
 '100000 people',
 '120',
 '150',
 '15th',
 '1989',
 '1995',
 '1st',
 '200',
 '200 people',
 '200000',
 '2003',
 '2009',
 '2011',
 '2011 ha',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '250',
 '300',
 '300 people',
 '350',
 '400',
 '406th',
 '406th protest',
 '500',
 '500 people',
 '5th',
 '600',
 '700',
 '800',
 'abandoned',
 'abdullah',
 'ablaze',
 'able',
 'abortion',
 'abuja',
 'abuse',
 'academic',
 'accept',
 'access',
 'accident',
 'accommodation',
 'accord',
 'according',
 'according local',
 'according police',
 'account',
 'accusation',
 'accuse',
 'accused',
 'accusing',
 'acronym',
 'act',
 'acting',
 'action',
 'activist',
 'activity',
 'actual',
 'actual number',
 'added',
 'adding',
 'additional',
 'address',
 'addressed',
 'addressing',
 'administration',
 'administrative

In [53]:
notes_clean_df=pd.DataFrame(notes_vc.todense(), 
                          columns=tf_vectorizer.get_feature_names())

In [54]:
notes_clean_df.shape

(14474, 3000)

In [55]:
notes_clean_df.columns

Index(['000', '000 000', '000 demonstrator', '000 people', '000 police',
       '000 protester', '000 student', '000 supporter', '000 worker', '000s',
       ...
       'yesterday morning', 'yesterday protest', 'yesterday thousand', 'young',
       'young men', 'young people', 'youth', 'yugoslavia', 'zimbabwe', 'zone'],
      dtype='object', length=3000)

In [56]:
# Add the vectorized text back to our original data frame
combined_df = pd.concat([df, notes_clean_df], axis =1)
combined_df.head(1)

,protestnumber,protesterviolence,participants,notes,accomodation,arrests,beatings,crowd dispersal,ignore,killings,...,yesterday morning,yesterday protest,yesterday thousand,young,young men,young people,youth,yugoslavia,zimbabwe,zone
0,1,0.0,3000,Canada railway passenger system was finally ...,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
combined_df.shape

(14474, 3222)

In [58]:
# Drop the original notes column
combined_df.drop(columns = 'notes', inplace=True)


In [59]:
# Write to csv
combined_df.to_csv('../data/numeric_text_combined_state_response.csv', index=False)